In [1]:
from __future__ import annotations

from qamsi.strategies.scaling.volatility_targeting import VolatilityTargeting
from qamsi.strategies.heuristics.equally_weighted import EWStrategy
from qamsi.cov_estimators.cov_estimators import CovEstimators
from qamsi.utils.data import read_csv
from run import Dataset

REBAL_FREQ = "ME"
TOP_N = 30
DATASET = Dataset.TOPN_US
ESTIMATOR = CovEstimators.RF.value(shrinkage_type="linear")
BASELINE = EWStrategy()

In [2]:
strategy_name = DATASET.name + ESTIMATOR.__class__.__name__ + f"_rebal{REBAL_FREQ}"
baseline_name = DATASET.name + BASELINE.__class__.__name__ + f"_rebal{REBAL_FREQ}"

strategy = read_csv(DATASET.value(topn=TOP_N).SAVE_PATH, strategy_name + ".csv")
baseline = read_csv(DATASET.value(topn=TOP_N).SAVE_PATH, baseline_name + ".csv")

In [3]:
strategy_excess_r = strategy["strategy_xs_r"]
baseline_excess_r = baseline["strategy_xs_r"]
rf = strategy["acc_rate"]

In [4]:
rebal_dates = strategy["rebal"]
rebal_dates = rebal_dates[rebal_dates].index

In [5]:
vol_scaler = VolatilityTargeting(
    trading_lag=1,
    max_exposure=2,
)
scaled_strategy_total_r = vol_scaler(
    strategy_excess_r=strategy_excess_r,
    rebal_dates=rebal_dates,
    baseline=baseline_excess_r,
).add(rf, axis=0)

In [6]:
from qamsi.backtest.assessor import Assessor

assessor = Assessor(
    rf_rate=rf,
    factors=strategy[strategy.columns.difference(["strategy_xs_r", "rebal", "acc_rate"])],
)

In [7]:
assessor(scaled_strategy_total_r)

StrategyStatistics:
* final_nav = 27.986620
* geom_avg_total_r = 0.191527
* geom_avg_xs_r = 0.183748
* std_xs_r = 0.211951
* min_xs_r = -0.123799
* max_xs_r = 0.204718
* skew = 0.485863
* kurtosis = 28.270639
* max_dd = -0.439075
* sharpe = 0.866937
* alpha_buy_hold = 0.113956
* tracking_error_buy_hold = 0.151717
* ir_buy_hold = 0.751108
* factor_loadings = {'low_risk': 0.7077996382753247, 'momentum': -0.11930958962233715, 'quality': 0.19883618948744358, 'size': 0.0258793306144856, 'spx': 1.0250981529462693, 'value': -0.26581120016526893}
* alpha_benchmark = 0.090716
* alpha_benchmark_pvalue = 0.002314
* tracking_error_benchmark = 0.123707
* ir_benchmark = 0.733311
* ttest_pval = 0.074911
* levene_pval = 0.043881
* timing_ability_coef = -0.009171
* timing_ability_pval = 0.684426